In [2]:
# Altera caminho para manter padrão com Jupyter Book
%cd chapters/05/02/

/home/danielavancini/Projects/eng-analytics/chapters/05/02


## 5.2 Juntando dados com JOIN e UNION

Consultar  informações de diferentes tabelas é uma das tarefas mais comuns na análise de dados. Por exemplo, podemos querer o total de vendas (tabela pedidos) por produtos com preço acima de 30 (tabela produto). Ou simplesmente saber o nome dos clientes que fizeram um pedido, etc. Como é comum em SQL, não há uma única forma de juntar dados (chamado de JOIN) entre tabelas mas algumas maneiras que podemos utilizar a depender do que queremos responder. A forma mais simples é fazendo algumas pequenas alterações na consulta que usamos anteriormente, adicionando um prefixo para explicar ao banco de dados de qual tabela queremos retornar cada campo e uma lógica para dizer como juntá-las:

```sql
SELECT <tabela_1.nome_do_campo1, tabela2.nome_do_campo2,...> 
FROM tabela_1, tabela2
WHERE tabela_1.nome_do_campo1 = tabela_2.nome_do_campo1
```
Vamos utilizar essa nova estrutura para consultar o nome dos clientes que fizeram pedidos na nossa loja. Nessa hora é importante utilizar o diagrama ER de nosso banco de dados para verificar que a coluna CustomerId da tabela Order é uma chave estrangeira para a coluna Id da tabela Customer e portanto é uma boa candidata à ser utilizada como chave em nosso JOIN:

In [3]:
# Vamos utilizar a extensão ipython-sql junto com o pacote SQLAlchemy para rodar consultas SQL diretamente
from sqlalchemy import create_engine
path_data = '../../assets/data/'
%load_ext sql
%config SqlMagic.displaycon=False
%config SqlMagic.feedback=False

# Agora dizemos para a extensão do ipython-sql onde estão os dados
%sql sqlite:///../../../assets/data/northwind.sqlite

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [4]:
%%sql

SELECT 'Order'.Id,'Customer'.CompanyName,'Order'.OrderDate
FROM 'Order', 'Customer' 
WHERE 'Order'.CustomerId='Customer'.Id
LIMIT 10

Id,CompanyName,OrderDate
10248,Vins et alcools Chevalier,2012-07-04
10249,Toms Spezialitäten,2012-07-05
10250,Hanari Carnes,2012-07-08
10251,Victuailles en stock,2012-07-08
10252,Suprêmes délices,2012-07-09
10253,Hanari Carnes,2012-07-10
10254,Chop-suey Chinese,2012-07-11
10255,Richter Supermarkt,2012-07-12
10256,Wellington Importadora,2012-07-15
10257,HILARION-Abastos,2012-07-16



Em termos um pouco mais técnicos, a consulta que fizemos acima é chamada de INNER JOIN. Essa operação funciona como uma espécie de filtro, trazendo todos os registros das tabelas juntadas que estejam presentes em ambas as tabelas. No entanto, se quisermos outros comportamentos em nossa consulta precisamos utilizar expressamente o comando JOIN  e entender um pouco melhor como ele funciona. Para isso recorremos à teoria dos conjuntos e ao diagrama de Venn:
* **(INNER) JOIN:** Retorna os campos onde há correspondência em ambas as tabelas
* **LEFT (OUTER) JOIN**: Retorna todos os campos da tabela da esquerda, e os campos com correspondência na tabela da direita
* **RIGHT (OUTER) JOIN**: Retorna todos os campos da tabela da direita, e os campos com correspondência na tabela da esquerda
* **FULL (OUTER) JOIN**: Retorna todos os campos da tabela da esquerda e da direita

![](../../../assets/img/db_venn.png)

Além do INNER JOIN que já mostramos anteriormente,  a operação de LEFT JOIN é a mais utilizada em consultas analíticas porque ela adiciona campos de uma outra tabela sem afetar a tabela inicial, semelhante à uma função PROCV() em planilhas eletrônicas. Para realizar um JOIN qualquer precisamos descrever qual tipo de JOIN queremos fazer e uma ou mais campos que devem ser correspondidos entre as tabelas:

```sql
SELECT <tabela_1.nome_do_campo1, tabela2.nome_do_campo2,...> 
FROM tabela_1
(INNER | LEFT| RIGHT |FULL) JOIN tabela_2 ON tabela_1.campo1 = tabela_2.campo2
(AND tabela_1.campo2 = tabela_2.campo2, ...)
```

Ao realizar um JOIN nós estamos fazendo uma operação vertical sobre as tabelas, isto é, estamos unindo diferentes colunas entre si a partir de uma relação de correspondência. Em alguns casos, nosso objetivo pode ser de unir duas consultas com as mesmas colunas, aumentando o número de linhas. Esse tipo de operação é feito através do comando UNION:

```sql
SELECT campo1, campo2
FROM tabela_1
UNION
SELECT campo1, campo2
FROM tabela_2
```

Se quiséssemos, por exemplo, criar uma consulta com todos os clientes e fornecedores do Brasil em uma mesma tabela, poderíamos usar uma UNION:

In [13]:
%%sql
SELECT Id, CompanyName
FROM  'Customer' 
WHERE Country = 'Brazil'
UNION
SELECT Id, CompanyName
FROM  'Supplier' 
WHERE Country = 'Brazil'

 * sqlite:///../../../assets/data/northwind.sqlite
Done.


Id,CompanyName
10,Refrescos Americanas LTDA
COMMI,Comércio Mineiro
FAMIA,Familia Arquibaldo
GOURL,Gourmet Lanchonetes
HANAR,Hanari Carnes
QUEDE,Que Delícia
QUEEN,Queen Cozinha
RICAR,Ricardo Adocicados
TRADH,Tradição Hipermercados
WELLI,Wellington Importadora
